# Effective Sample Size in 5 columns

Effective sample size is given by $n_{\text{eff}}=\frac{n}{1+(m-1)\rho}$ where $n$ is the sample size, $m$ is the average group size, and $\rho$ is correlation (in the intraclass correlation sense). We are working with data which can be grouped along both ensemble member and (lat,lon) pair. Therefore, as a first pass I will calculate $\rho$ by considering each ensemble member to be its own group (80 groups). 

$$r={\frac {K}{K-1}}\cdot {\frac {N^{-1}\sum _{n=1}^{N}({\bar {x}}_{n}-{\bar {x}})^{2}}{s^{2}}}-{\frac {1}{K-1}}$$
where $K$ is the number of data values per group, and ${\bar {x}}_{n}$ is the sample mean of the $n$th group.



### Import packages and define functions

In [1]:
import numpy as np
import xarray as xr

In [2]:
def ess(sample_size, group_size, rho):
    n_eff = sample_size/( 1 + (group_size-1)*rho)
    return n_eff

In [3]:
def intraclass_correlation(arrayy):
#arrayy is a 2-dimensional array with each row holding values corresponding to a single group
    N = arrayy.shape[0] # number of groups
    K = arrayy.shape[1] # number of data values per group
    xbar = np.mean(arrayy)
    s2 = np.var(arrayy)
    grp_mean = np.mean(arrayy, axis=1)
    grp_mean_centered = grp_mean - xbar
    grp_var = np.mean(np.square(grp_mean_centered))
    r = (K / (K-1)) * (grp_var / s2) - 1/(K-1)
    return r

In [4]:
def compute_ess_from_array(arrayy):
    sample_size = arrayy.size    # total number of samples
    group_size = arrayy.shape[1] # number of data values per group
    corr = intraclass_correlation(arrayy)
    n_eff = ess(sample_size, group_size, corr)
    return n_eff

### A very synthetic example

Let's contruct an array with 3 groups and 2 elements per group. The groups appear to be 'clustered', and hence we should see a large correlation coefficient.

In [5]:
x = np.array([[1,2],[23,27],[7, 13]])
print(x)

[[ 1  2]
 [23 27]
 [ 7 13]]


In [6]:
intra_corr = intraclass_correlation(x)
print('Intraclass correlation is large: '+str(intra_corr))

Intraclass correlation is large: 0.9105988192296881


In [7]:
print('Effective sample size is close to the number of groups: '+str(compute_ess_from_array(x)))

Effective sample size is close to the number of groups: 3.1403766921718654


### Now consider SST in the Tropical Pacific

#### Load data

In [8]:
## Where are we working
proj_dir = '/Users/zstanley/Documents/git_repos/obs_loc_for_scda'
plot_dir = proj_dir + '/plots/five_columns_error_in_kalman_gain/'
my_data_dir = proj_dir + '/my_data/20151206.030000'
nb_dir = proj_dir + '/notebooks'

In [9]:
these_columns = {
  'lons' : [-154.5, 35.5, 75.5, -150.5, 160.5],
  'lats' : [-27.5, -49.5, -31.5, 12.5, 40.5],
  'name' : ['South Pacific', 'Southern Ocean', 'Indian Ocean', 'Tropical Pacific', 'North Pacific'],
  'save_name' : ['south_pacific2', 'southern_ocean2', 'indian_ocean2', 'tropical_pacific2', 'north_pacific']
}

In [10]:
## Load vertical columns
south_pacific = xr.open_dataset(my_data_dir+'/five_columns_'+these_columns['save_name'][0]+'.nc')
southern_ocean = xr.open_dataset(my_data_dir+'/five_columns_'+these_columns['save_name'][1]+'.nc')
indian_ocean = xr.open_dataset(my_data_dir+'/five_columns_'+these_columns['save_name'][2]+'.nc')
tropical_pacific = xr.open_dataset(my_data_dir+'/five_columns_'+these_columns['save_name'][3]+'.nc')
north_pacific = xr.open_dataset(my_data_dir+'/five_columns_'+these_columns['save_name'][4]+'.nc')

In [68]:
ds = north_pacific

In [69]:
num_groups = ds['ens_mem'].size
num_per_group = ds['lat'].size * ds['lon'].size
sst = np.empty((num_groups, num_per_group))
ast = np.empty((num_groups, num_per_group))
ind = 0
for lat in ds['lat'].values:
    for lon in ds['lon'].values:
        # sst
        hold = ds['sst'].sel(lat=lat, lon=lon)
        hold = hold - hold.mean('ens_mem')
        hold = hold.to_numpy()
        sst[:, ind] = hold
        # ast
        hold = ds['atm_T'].sel(atm_lev=126, lat=lat, lon=lon)
        hold = hold - hold.mean('ens_mem')
        hold = hold.to_numpy()
        ast[:, ind] = hold
        ind = ind + 1

In [70]:
print('Effective sample size for AST is: '+str(compute_ess_from_array(ast)))
print('Effective sample size for SST is: '+str(compute_ess_from_array(sst)))

Effective sample size for AST is: 206.3017495541281
Effective sample size for SST is: 608.0227181984432
